<a href="https://colab.research.google.com/github/ioahKwon/Machine-Learning-Deep-Learning/blob/master/%5B%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%EA%B3%BC%EB%94%A5%EB%9F%AC%EB%8B%9D%5D_Week_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[머신러닝과딥러닝] 수업 정리 Week #11**

### _2020.08._ SKKU Lecture Summary by IOAH


---





### 1. Dropout을 포함한 Deep Neural Network 설계

(MLP를 이용한 MNIST DATASET 분류)


오늘 실습에서는 신경망의 크기를 변화시켜가며 overfitting 여부를 확인하고 dropout을 적용하여 overfitting을 해결해볼 것이다. 우선 전체적인 프로세스는 MNIST 데이터를 로드하고 이 데이터를 전처리 한 다음 모델을 생성하고 변수를 초기화한다. 그 후 주어진 Train data에 맞춰 모델을 학습시키고 test data를 통해 모델을 평가한다. 우선 필수적인 module을 import 해보도록 하자. 각각 데이터 로드, 모델 생성 및 초기화 그리고 데이터 처리에 대한 module 이다.

In [ ]:
from keras.datasets import mnist
from keras import models
from keras import layers
from keras.utils import to_categorical

Using TensorFlow backend.


그 후 데이터를 로드해 주는데 MNIST 데이터는 0~9 사이의 숫자 데이터며 이미지 파일이다.

 로드한 후 로드한 데이터를 사이즈 조절해주고 SCALE을 맞춰준다. 대부분 0~255의 Scale을 가지므로 255로 나눠준다. 그 후 y값인 labels 값을 to_categorical 함수를 통해 범주형 변수로 변경해준다.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
train_images = train_images.reshape((60000,28*28))
train_images = train_images.astype('float32')/ 255
test_images = test_images.reshape((10000,28*28))
test_images = test_images.astype('float32')/ 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

다음은 모델을 생성하고 초기화하는 부분이다. 모델 구조를 sequential함수를 통해 선언해주고 Hidden layer의 output 노드 수는 512개로, activation function은 ‘relu’로, 그리고 마지막 output layer의 노드 수는 10으로 고정해준다. 여기서 10으로 고정해주는 이유는 MNIST 데이터가 0~9까지의 숫자니까 Class가 10개이기 때문이다. 그 다음 모델의 optimizer를 rmsprop으로, loss function은 categorical_crossentropy로 주며 평가방식은 accuracy로 하기로 한다. Model.summary()를 통해 본 모델의 구조는 아래와 같다.

In [ ]:
model = models.Sequential()
model.add(layers.Dense(512,activation='relu', input_shape=(28*28,)))
model.add(layers.Dense(10,activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


epoch을 10을 주고 batch_size는 128로 지정하여 모델을 학습시킨다. 밑의 결과를 보면 train set에 대해서는 accuracy가 무려 92.22%인 것을 확인할 수 있다. 또한 학습이 진행됨에 따라 loss는 줄어들고 accuracy는 증가하는 것을 볼 수 있다. 그리고 test set에 대해선 accuracy가 약 92.8%인 것을 확인할 수 있다. 

In [ ]:
##epoch수 변경으로 모델의 학습을 조절
model.fit(train_images, train_labels, epochs=10, batch_size=128)



Epoch 1/10
60000/60000 [==============================] - 5s 79us/step - loss: 1.4985 - accuracy: 0.6584
Epoch 2/10
60000/60000 [==============================] - 4s 74us/step - loss: 0.6030 - accuracy: 0.8506
Epoch 3/10
60000/60000 [==============================] - 4s 75us/step - loss: 0.4189 - accuracy: 0.8871
Epoch 4/10
60000/60000 [==============================] - 5s 75us/step - loss: 0.3573 - accuracy: 0.8999
Epoch 5/10
60000/60000 [==============================] - 5s 75us/step - loss: 0.3266 - accuracy: 0.9067
Epoch 6/10
60000/60000 [==============================] - 5s 78us/step - loss: 0.3065 - accuracy: 0.9118
Epoch 7/10
60000/60000 [==============================] - 5s 76us/step - loss: 0.2918 - accuracy: 0.9157
Epoch 8/10
60000/60000 [==============================] - 4s 74us/step - loss: 0.2795 - accuracy: 0.9193
Epoch 9/10
60000/60000 [==============================] - 4s 74us/step - loss: 0.2683 - accuracy: 0.9222
Epoch 10/10
60000/60000 [==============================

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 0s 47us/step


0.9279000163078308

이번에는 overfitting을 막기 위한 방법 중 하나인 Dropout을 적용시켜보자. 모델에 dropout layer를 추가해주고 dropout rate는 0.2로 설정해준다. 즉, 랜덤으로 20%만큼 weight를 제외하고 학습을 돌린다는 것이다. 결과는 아래와 같다.

In [ ]:
model2 = models.Sequential()
model2.add(layers.Dense(512,activation='relu', input_shape=(28*28,)))
model2.add(layers.Dropout(0.2,noise_shape=None, seed = None))
model2.add(layers.Dense(10,activation='softmax'))
model2.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model2.fit(train_images, train_labels, epochs=10, batch_size=128)

Epoch 1/10
60000/60000 [==============================] - 5s 84us/step - loss: 1.5129 - accuracy: 0.6470
Epoch 2/10
60000/60000 [==============================] - 5s 83us/step - loss: 0.6214 - accuracy: 0.8427
Epoch 3/10
60000/60000 [==============================] - 5s 84us/step - loss: 0.4348 - accuracy: 0.8803
Epoch 4/10
60000/60000 [==============================] - 5s 81us/step - loss: 0.3706 - accuracy: 0.8953
Epoch 5/10
60000/60000 [==============================] - 5s 86us/step - loss: 0.3373 - accuracy: 0.9031
Epoch 6/10
60000/60000 [==============================] - 5s 83us/step - loss: 0.3147 - accuracy: 0.9088
Epoch 7/10
60000/60000 [==============================] - 5s 86us/step - loss: 0.2968 - accuracy: 0.9133
Epoch 8/10
60000/60000 [==============================] - 5s 84us/step - loss: 0.2808 - accuracy: 0.9176
Epoch 9/10
60000/60000 [==============================] - 5s 81us/step - loss: 0.2684 - accuracy: 0.9221
Epoch 10/10
60000/60000 [==============================

In [ ]:
test_loss, test_acc = model2.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 0s 46us/step


0.9290000200271606

원래 의도대로라면 dropout 추가 후 train data의 accuracy는 줄어들고 test data의 accuracy는 증가해야 하지만 현재 상황에선 dropout 없는 모델과 dropout을 추가한 모델 둘은 거의 비슷한 성능을 보이는 것으로 보인다. (사실상 둘이 거의 비슷하다고 봐도 뭐...ㅋㅋㅋㅋㅋ)

그래도 보면 그나마 dropout layer가 있을 때가 아~~주 소폭 성능이 향상된 것을 볼 수 있다.

즉, drop out layer의 추가 후 overfitting이 조금 해소되어 더 좋은 성능을 보인다는 것을 확인할 수 있다.